In [1]:
from PIL import Image
import matplotlib.pyplot as plt
import math
import os
import random
import torch
from torch import nn
import numpy as np
from torchvision import transforms



DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{DEVICE=}")


DEVICE='cuda'


In [2]:
!bash dataset_fetch.sh

+ '[' '!' -d dataset ']'
+ echo 'Dataset Directory already exist!'
Dataset Directory already exist!


In [3]:
# Helper Cell


def helper_imageGrid(image_paths, cols=4):
    img_count = len(image_paths)
    rows = math.ceil(img_count / cols)
    fig, axes = plt.subplots(rows, cols, figsize=(20, 20))
    irow, icol = 0, 0
    for path in image_paths:
        ax = axes[irow][icol]
        ax.imshow(Image.open(path))
        title = path.split("/")[-2]
        #ax.title(path)
        ax.axis("off")

        icol += 1
        if icol % cols == 0:
            icol = 0
            irow += 1

        
    
    return fig

import re


def helper_fetch_localmodel(mname="autoencoder", device=DEVICE):
    max_name = None
    max_num = 0
    for x in os.listdir():
        if a := re.match(f"{mname}_\\d+", x):
            name = a.group()
            num = int(name.split("_")[-1])
            max_num = num if max_num < num else max_num
    if max_num > 0:
        return max_num, torch.load(f"model/{mname}_{max_num}", map_location=torch.device(device))
    
def helper_save_model(model, epoch):
    try:
        name = f"{model._get_name()}_{epoch}"
        torch.save(model.state_dict(), name)
        return True
    except Exception as e:
        print("[ERROR]: Some Error Occured!!", e)
    return False

    
    


In [4]:


image_paths = []
for i, x in enumerate(os.walk("dataset/GAID")):
    parent, subdir, files = x
    if len(subdir) != 0: continue

    image_paths += [ f"{parent}/{file}" for file in files ]


sizes = set()
for x in image_paths:
    img = Image.open(x)
    sizes.add(img.size)
    img.close()


print("Sizes of Images: ", sizes)
print("Count of Images: ", len(image_paths))




Sizes of Images:  {(300, 300)}
Count of Images:  20000


In [19]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


REPRESENTATION_DIM = 100




samples = []
pca = PCA(n_components=REPRESENTATION_DIM)
scaler = StandardScaler()

for path in image_paths:
    im = Image.open(path)
    data = np.uint8(im).flatten()
    samples += [ data ]

    if len(samples) % 1000 == 0 :
        ssamples = scaler.fit_transform(samples)
        pca.fit(ssamples)
        del ssamples
        del samples
        samples = []
        print("Batch is Done!!")


    im.close()











SyntaxError: invalid syntax (504270257.py, line 21)

In [21]:
import pickle

pca = pickle.load(open("pca_model", "rb"))
scaler = pickle.load(open("scaler_model", "rb"))

In [7]:
del samples

In [8]:
def read_image(path):
    return np.float32(Image.open(path)) / 255

def encode_image(path):
    im = read_image(path)
    data = im.flatten().reshape(1, -1)
    data = scaler.transform(data)
    return im, pca.transform(data)

In [11]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.decoder = nn.Sequential(
            nn.Unflatten(2, (10, 10)),
            nn.ConvTranspose2d(in_channels=1, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.UpsamplingNearest2d(None, (3, 3)),
            nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.UpsamplingNearest2d(None, (3, 3)),
            nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.UpsamplingNearest2d(None, (3, 3)),
            nn.ConvTranspose2d(in_channels=64, out_channels=3, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.UpsamplingNearest2d((300, 300)),
            # nn.Sigmoid()
        )

    def forward(self, x):
        return self.decoder(x)
    
def prepareTensor(encoded_image):
    return torch.tensor(encoded_image, dtype=torch.float32)



NUM_EPOCHS=-1
LR=0.01


print(f"{DEVICE}, Device used")

model = Decoder().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
mse = nn.MSELoss()




cpu, Device used


In [22]:
class ConvDataset2(torch.utils.data.Dataset):

    def __init__(self, paths):
        self.paths = paths

    def __getitem__(self, index):
        o_img, e_img = encode_image(self.paths[index])
        o_img = torch.tensor(np.moveaxis(o_img, [0, 1, 2], [1, 2, 0]))
        e_img = prepareTensor(e_img)
        return o_img, e_img
    
    def __len__(self):
        return len(self.paths)




dataset = ConvDataset2(image_paths)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=20, shuffle=True)

In [ ]:
epoch = 0
try:
    while epoch != NUM_EPOCHS:
        optimizer.zero_grad()
        for data in dataloader:
            o_img, sample = data
            sample = sample.to(DEVICE)
            o_img = o_img.to(DEVICE)
            out = model(sample)
            loss = mse(out, o_img)
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch}/{NUM_EPOCHS}, {loss.item()}")
        epoch += 1
except Exception as e:
    pass
finally:
    helper_save_model(model, epoch)


[ERROR]: Some Error Occured!! 'Decoder' object has no attribute '_get_name_'


KeyboardInterrupt: 